<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/evaluate_models/ATE_studio_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

     |████████████████████████████████| 296kB 7.4MB/s 
     |████████████████████████████████| 1.2MB 12.9MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 481kB 26.8MB/s 
     |████████████████████████████████| 143kB 32.0MB/s 
     |████████████████████████████████| 112kB 34.0MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for alembic: filename=alembic-1.5.5-py2.py3-none-any.whl size=156597 sha256=93bdd24e2bd6a3b432d8b7b2d28fd2ce758a526d6f590dbf78f35cbab9988593
  Stored in directory: /root/.cache/pip/wheels/74/3f/61/7de6e3cef766d2680a5d81b1a388286e640f6a681eb589d643
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=5aff7c646cdcdb83ba5b85cf984e2dfe8e1ee389399a3288e360ccaf7f3122fc
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256

In [3]:
import numpy as np
import pandas as pd
import pickle
import optuna
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [4]:
path="/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/"

In [5]:
test = pd.read_json(path+'ATE_ABSITA_test_set/ate_absita_gold.ndjson'
                       , lines=True)

train = pd.read_json(path+'ATE_ABSITA_training_set/ate_absita_training.ndjson'
                       , lines=True)

In [6]:
print(test.columns)
print(train.columns)

Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')
Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')


In [7]:
train.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
test.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
print(f'Contains {len(train)} sentences')
print(f'Contains {len(test)} sentences')

Contains 3054 sentences
Contains 1200 sentences


Creazione colonna Positivi/Negativi

In [8]:
train["review_type"] = train["score"].apply(lambda x: "neg" if x < 5 else "pos")
test["review_type"] = test["score"].apply(lambda x: "neg" if x < 5 else "pos")

print(f'TRAIN::\n{train.review_type.value_counts()}')
print(f'TEST::\n{test.review_type.value_counts()}')

TRAIN::
pos    2150
neg     904
Name: review_type, dtype: int64
TEST::
pos    857
neg    343
Name: review_type, dtype: int64


Rimozione Colonna Score in quanto non piu significativa per la Sentiment Analysis

In [9]:
train.drop(columns=['score'], inplace=True)
test.drop(columns=['score'], inplace=True)

In [10]:
with open(path+"word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Trasformazione input da frasi a vettori di parole

In [11]:
def my_text_to_word_sequence(sentence):
  return keras.preprocessing.text.text_to_word_sequence(sentence,
                                                        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`\'{|}~\t\n',
                                                        lower=True)

TRAIN, Encoding e Dataset per modello

In [12]:
one_hot_train = OneHotEncoder(sparse=False).fit_transform(
        train.review_type.to_numpy().reshape(-1, 1)
        )

In [13]:
sentences = [my_text_to_word_sequence(sentence) for sentence in train['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [15]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)

print(f'Il massimo è {max}')

Il massimo è 85


Crao i vari embedding per tutti i dataset, quest'operazione e pesante 

In [16]:
embedded_trainset = np.zeros(shape=(len(sentences), max, 300))
for i, sentence in enumerate(sentences):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

Modello

In [17]:
best_params = optuna.load_study(study_name="ATE", storage="sqlite:///"+path+"optuna_ATE_studio_0.db").best_params

In [18]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params["units"],
                                                             recurrent_dropout=best_params["dropout"],
                                                             activation='tanh')))
model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.Adam(0.001),
            metrics=['accuracy'])

In [ ]:
#train model
result = model.fit(embedded_trainset,
                 one_hot_train,
                 epochs=100,
                 batch_size=best_params["batch_size"],
                 callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

In [ ]:
#save the model
model.save_weights(path+'ATE_w_studio_0.h5')

In [ ]:
#load only the w of the model, the model must be already executed
model.load_weights(path+'ATE_w_studio_0_lorenzo.h5')

EVALUATION

In [ ]:
model.evaluate(embedded_trainset,one_hot_train)